In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', None)  # Show all column content
pd.set_option('display.max_columns', None)  # Show all columns

In [3]:
df = pd.read_csv("reddit_posts_comments_Depression.csv")

In [4]:
df_post = df['Post Title']
df_body = df['Post Body']
df_comment = df['Comment']

In [5]:
df_post = pd.DataFrame(df_post)
df_post.reset_index(drop=True)
print()

In [6]:
df_body = pd.DataFrame(df_body)
df_body.reset_index(drop=True)
print()

In [7]:
df_comment = pd.DataFrame(df_comment)
df_comment.reset_index(drop=True)
print()

In [8]:
from warnings import filterwarnings
filterwarnings(action='ignore')

In [9]:
df_post['Post Title'] = df_post['Post Title'].apply(lambda m: m.strip().lower())
df_body['Post Body'] = df_body['Post Body'].apply(lambda m: m.strip().lower())
df_comment['Comment'] = df_comment['Comment'].apply(lambda m: m.strip().lower())

In [10]:
import re

# Function to clean and retain only words (strings)
def clean_text(x):
    if isinstance(x, str):  # Ensure it's a string
        # Use regex to extract words, join them with spaces, and convert to lowercase
        return " ".join(re.findall(r'\b\w+', x))
    return None  # Discard non-strings

# Apply the cleaning function to the DataFrame columns
df_post['Post Title'] = df_post['Post Title'].apply(clean_text)
df_body['Post Body'] = df_body['Post Body'].apply(clean_text)
df_comment['Comment'] = df_comment['Comment'].apply(clean_text)


In [11]:
df_post['Post Title'] = df_post['Post Title'].apply(lambda text: re.sub(r'\s+', ' ',re.sub(r'_', ' ', re.sub(r'\d+', '', text))).strip())

df_body['Post Body'] = df_body['Post Body'].apply(lambda text: re.sub(r'\s+', ' ',re.sub(r'_', ' ', re.sub(r'\d+', '', text))).strip())

df_comment['Comment'] = df_comment['Comment'].apply(lambda text: re.sub(r'\s+', ' ',re.sub(r'_', ' ', re.sub(r'\d+', '', text))).strip())

## Tokenization

In [12]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TreebankWordTokenizer

In [13]:
tokenizer = TreebankWordTokenizer()

In [14]:
import nltk
# nltk.download('punkt')

In [15]:
df_post['Post Title'] = df_post['Post Title'].apply(tokenizer.tokenize)
df_comment['Comment'] = df_comment['Comment'].apply(tokenizer.tokenize)

In [16]:
from nltk.corpus import stopwords

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\navne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop_words = stopwords.words('english')

In [19]:
def remove_stopwords(message):
    global stop_words
    result = []
    for word in str(message).split():  # Ensure the input is a string
        if word.lower().strip() not in stop_words:
            result.append(word)
    return " ".join(result)

# Apply the function to each entry in the Post Body column
df_post['Post Title'] = df_post['Post Title'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)  # Convert lists to strings
df_post['Post Title'] = df_post['Post Title'].apply(remove_stopwords)

df_comment['Comment'] = df_comment['Comment'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)  # Convert lists to strings
df_comment['Comment'] = df_comment['Comment'].apply(remove_stopwords)

In [20]:
# pip install spacy

In [21]:
# !python -m spacy download en_core_web_sm

In [22]:
# pip install numpy==1.26.4

In [23]:
df_post['Post Title'] = df_post['Post Title'].apply(tokenizer.tokenize)
df_comment['Comment'] = df_comment['Comment'].apply(tokenizer.tokenize)

In [24]:
df_comment.head(3)

,Comment
0,"[understood, apologise, forget, future, break]"
1,[removed]
2,"[biggest, problem, private, talks, may, may, reach, point, advice, one, needs, help, leave, chat, would, even, make, worse, public, chat, good, chance, others, may, jump, got, lost]"


In [25]:
df_comment = df_comment[df_comment["Comment"].apply(lambda x: x != ["removed"])]

In [26]:
df_comment = df_comment[df_comment["Comment"].apply(lambda x: x != ["deleted"])]

In [27]:
df_comment.reset_index(drop=True)
print()

In [28]:
df_post = df_post.drop_duplicates()

In [29]:
df_post = df_post.reset_index(drop=True)
print()

In [30]:
import nltk
# nltk.download('averaged_perceptron_tagger')

In [31]:
import pandas as pd
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re
import nltk

# nltk.download('wordnet')
# nltk.download('omw-1.4')

def get_wordnet_pos(word):
    """
    Map POS tag to first character lemmatize() accepts.
    Only adjectives (a) and verbs (v) are considered.
    """
    tag = wordnet.synsets(word)
    if tag:
        pos = tag[0].pos()
        if pos in {'a', 'v'}:  # Adjective or Verb
            return pos
    return None

def clean_text(text):
    """
    Perform text cleaning including lemmatization, filtering, and deduplication.
    Always keep certain whitelisted words.
    """
    lemmatizer = WordNetLemmatizer()

    # Ensure input is a string
    if not isinstance(text, str):
        text = str(text)

    # Tokenize text and remove non-alphabetic words
    words = re.findall(r'\b[a-zA-Z]+\b', text)

    # Define a whitelist of words to always keep
    whitelist = {'depression', 'affect', 'psychiatrist', 'suicide', 'worse','sad','disturbing','mental','commit','broken','break'}

    # Lemmatize and filter adjectives and verbs
    lemmatized_words = []
    for word in words:
        pos = get_wordnet_pos(word)
        lemma = lemmatizer.lemmatize(word.lower(), pos) if pos else word.lower()
        if pos and lemma not in whitelist:  # Keep only adjectives and verbs unless in whitelist
            lemmatized_words.append(lemma)

    # Add whitelisted words explicitly
    lemmatized_words.extend(word for word in words if word.lower() in whitelist)

    # Remove duplicates and words differing by a single letter
    unique_words = set()
    final_words = []
    for word in lemmatized_words:
        if not any(len(word) == len(other) and sum(c1 != c2 for c1, c2 in zip(word, other)) == 1 for other in unique_words):
            unique_words.add(word)
            final_words.append(word)

    return ' '.join(sorted(set(final_words)))

# Convert all entries in 'Post Title' to strings and apply the cleaning function
df_post['Post Title'] = df_post['Post Title'].astype(str)
df_post['Post Title'] = df_post['Post Title'].apply(clean_text)

df_comment['Comment'] = df_comment['Comment'].astype(str)
df_comment['Comment'] = df_comment['Comment'].apply(clean_text)

# combining into single data frame
# df = pd.concat([df_post['Post Title'], df_comment['Comment']], axis=1)

# Rename columns for clarity
# df.columns = ['Post Title', 'Comment']

# save the cleaned dataset to a new CSV file

# df.to_csv('cleaned_dataset.csv', index=False)

In [32]:
df_post.describe

<bound method NDFrame.describe of                              Post Title
0    broken explain make new understand
1                                      
2             make psychiatrist suicide
3                                      
4                     affect depression
..                                  ...
301                                    
302                                fail
303                                pour
304                                    
305                                want

[306 rows x 1 columns]>